In [ ]:
# 📦 필요 모듈
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files

# 1️⃣ 파일 업로드
print("📂 인사대장 파일을 업로드하세요.")
uploaded1 = files.upload()
file1_name = list(uploaded1.keys())[0]

print("📂 전자계약서 업로딩 파일을 업로드하세요.")
uploaded2 = files.upload()
file2_name = list(uploaded2.keys())[0]

# 2️⃣ 엑셀 파일 로드
df1 = pd.read_excel(file1_name, sheet_name="Sheet1")             # 기존 계약 데이터
salary_table = pd.read_excel(file1_name, sheet_name="Sheet2")    # 밴드별 급여
location_table = pd.read_excel(file1_name, sheet_name="Sheet3")  # 근무지코드 → 근무장소 테이블
df2 = pd.read_excel(file2_name, sheet_name="Sheet1")             # 신규 입사자 데이터

# 3️⃣ 입사일 기준 필터
df_filtered = df2[df2['입사일'] == '2025-01-01'].copy()

# 4️⃣ 근무지코드와 근무장소 매핑 (Sheet3 기준: 첫 번째 열 = 코드, 두 번째 열 = 장소)
location_map = location_table.set_index(location_table.columns[0])[location_table.columns[1]].to_dict()
df_filtered["근무장소"] = df_filtered["근무지코드"].map(location_map)

# 5️⃣ 신규 데이터 생성
df_new = pd.DataFrame()
df_new["사번"] = df_filtered["사번"]
df_new["구분"] = df_filtered["구분"]
df_new["기준일자"] = "2025-01-01"
df_new["계약시작일"] = "2025-01-01"
df_new["계약종료일"] = (
    datetime.strptime("2025-01-01", "%Y-%m-%d") + timedelta(days=6*30)
).strftime("%Y-%m-%d")
df_new["근무장소"] = df_filtered["근무장소"]
df_new["밴드"] = df_filtered["밴드"]

# 6️⃣ 급여 정보 병합
df_merged = pd.merge(df_new, salary_table, on="밴드", how="left")

# 7️⃣ 기존 컬럼 기준 정리
columns_to_keep = df1.columns.tolist()
df_cleaned = df_merged[columns_to_keep].dropna(axis=1, how='all')

# 8️⃣ 기존 Sheet1과 병합
df_final = pd.concat([df1, df_cleaned], ignore_index=True)

# 9️⃣ 정리
if '밴드' in df_final.columns:
    df_final.drop(columns=['밴드'], inplace=True)
df_final = df_final.dropna(how='all')

# 🔟 저장
final_file = "최종_정리본.xlsx"
df_final.to_excel(final_file, sheet_name="Sheet1", index=False)
print(f"✅ 엑셀 자동화 완료! 저장된 파일명: {final_file}")
files.download(final_file)

📂 인사대장 파일을 업로드하세요.


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# 1️⃣ 네이버 SMTP 설정
EMAIL_SENDER = "hawk_ahn@naver.com"       # 네이버 이메일
EMAIL_PASSWORD = "extreme199201!"          # 네이버 비밀번호
EMAIL_RECEIVER = "2010241@skpsnm.com"           # 수신자 (아웃룩)

def send_naver_email():
    msg = MIMEMultipart()
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER
    msg["Subject"] = "📢 업무 자동화 알림 (네이버 → 아웃룩)"

    body = "오늘은 자동화된 엑셀 작업을 실행해야 하는 날입니다! 코랩에서 실행해주세요 😊"
    msg.attach(MIMEText(body, "plain"))

    try:
        server = smtplib.SMTP_SSL("smtp.naver.com", 465)
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        server.quit()
        print("✅ [메일 발송 완료] 아웃룩 메일(2010241@skpsnm.com)로 알림이 전송되었습니다.")
    except Exception as e:
        print("❌ [메일 발송 실패] 메일 전송에 실패했습니다.")
        print(f"🔍 오류 메시지: {e}")

# 2️⃣ 날짜 조건 확인 후 메일 발송
today = datetime.today().day
if today in [1, 2, 14, 16]:
    send_naver_email()

# 3️⃣ 파일 업로드
print("📂 '통합 문서1.xlsx' 파일을 업로드하세요.")
uploaded1 = files.upload()
file1_name = list(uploaded1.keys())[0]

print("📂 '통합 문서2.xlsx' 파일을 업로드하세요.")
uploaded2 = files.upload()
file2_name = list(uploaded2.keys())[0]

# 4️⃣ 엑셀 데이터 불러오기
df1 = pd.read_excel(file1_name, sheet_name="Sheet1")
salary_table = pd.read_excel(file1_name, sheet_name="Sheet2")
df2 = pd.read_excel(file2_name, sheet_name="Sheet1")

# 5️⃣ 입사일 기준 필터링
df_filtered = df2[df2['입사일'] == '2025-01-01'].copy()

# 6️⃣ 신규 데이터 생성
df_new = pd.DataFrame()
df_new["사번"] = df_filtered["사번"]
df_new["구분"] = df_filtered["구분"]
df_new["기준일자"] = "2025-01-01"
df_new["계약시작일"] = "2025-01-01"
df_new["계약종료일"] = (datetime.strptime("2025-01-01", "%Y-%m-%d") + timedelta(days=6*30)).strftime("%Y-%m-%d")
df_new["근무장소"] = df_filtered["근무장소"]
df_new["밴드"] = df_filtered["밴드"]

# 7️⃣ 급여 정보 병합
df_merged = pd.merge(df_new, salary_table, on="밴드", how="left")

# 8️⃣ 컬럼 정리 (기존 컬럼만 유지)
columns_to_keep = df1.columns.tolist()
df_cleaned = df_merged[columns_to_keep]

# 9️⃣ 병합
df_final = pd.concat([df1, df_cleaned], ignore_index=True)

# 🔟 밴드 제거 + 공백 제거
if '밴드' in df_final.columns:
    df_final.drop(columns=['밴드'], inplace=True)
df_final = df_final.dropna(how='all')

# 🔁 저장 및 다운로드
final_file = "최종_정리본.xlsx"
df_final.to_excel(final_file, sheet_name="Sheet1", index=False)

print(f"✅ 엑셀 자동화 완료! 저장된 파일명: {final_file}")
files.download(final_file)

# 자동화 처리 끝난 후, 무조건 알림 메일 보내기
send_naver_email()


In [ ]:
import pandas as pd
from datetime import datetime
from google.colab import files

# 📂 1. 파일 업로드
print("📂 '통합 문서3.xlsx' 파일을 업로드하세요.")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 📄 2. 데이터 로드
df = pd.read_excel(file_name, sheet_name="Sheet1")

# 🗓 3. 날짜 변환
df["연차시작일"] = pd.to_datetime(df["연차시작일"], errors="coerce")
df["연차종료일"] = pd.to_datetime(df["연차종료일"], errors="coerce")

# 📆 4. 오늘 기준 남은 일수 계산
today = datetime.today()
df["남은일수"] = (df["연차종료일"] - today).dt.days

# 🎯 5. 대상자 분류
df_3개월 = df[(df["남은일수"] > 30) & (df["남은일수"] <= 91)].copy()
df_1개월 = df[(df["남은일수"] > 0) & (df["남은일수"] <= 30)].copy()
df_소멸됨 = df[df["남은일수"] <= 0].copy()  # 이미 만료된 연차

# 📌 6. 소멸 예정 연차 = 잔여쿼터
for target in [df_3개월, df_1개월, df_소멸됨]:
    target["소멸예정연차"] = target["잔여쿼터"]

# 📝 7. 컬럼 정리
columns = ["사번", "이름", "연차시작일", "연차종료일", "잔여쿼터", "소멸예정연차", "남은일수"]
df_3개월 = df_3개월[columns]
df_1개월 = df_1개월[columns]
df_소멸됨 = df_소멸됨[columns]

# 💾 8. 엑셀 저장
output_file = "연차_소멸예정_대상자_최종.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df_3개월.to_excel(writer, sheet_name="3개월 이내", index=False)
    df_1개월.to_excel(writer, sheet_name="1개월 이내", index=False)
    df_소멸됨.to_excel(writer, sheet_name="이미 소멸됨", index=False)

# 📥 9. 다운로드
print("✅ 연차 소멸 예정 대상자 분석 완료! 엑셀 파일을 다운로드하세요.")
files.download(output_file)

📂 '통합 문서3.xlsx' 파일을 업로드하세요.


Saving 통합 문서3.xlsx to 통합 문서3 (1).xlsx
✅ 연차 소멸 예정 대상자 분석 완료! 엑셀 파일을 다운로드하세요.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>